In [ ]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

import cv2
import rospy
import rosbag
import numpy as np
import os, csv, time, argparse
from matplotlib import pyplot as plt

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from gazebo_msgs.msg import ModelStates
from sensor_msgs.msg import Image, CameraInfo
from cv_bridge import CvBridge, CvBridgeError
from tf.transformations import euler_from_quaternion, quaternion_from_euler

from src.VBOATS import VBOATS

In [ ]:
class image_converter:
    def __init__(self):
        rospy.init_node('vboat_pipeline')
        self.bridge = CvBridge()
        self.image_pub = rospy.Publisher("/obstacles",Image,queue_size=1000)
        self.image_sub = rospy.Subscriber("/ugv/d415/depth/image_raw",Image,self.callback)
        
        self.vboat = VBOATS()
        self.vboat.dead_x = 0
        self.vboat.dead_y = 5
        self.r = rospy.Rate(40)
        self.img = []
        self.obs_disp = []
        
    def callback(self,data):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(data, "16UC1")
            cv_image = np.float32(cv_image*0.001)
            self.img = np.copy(cv_image)
        except CvBridgeError as e:
            print(e)

        self.vboat.pipeline(np.uint8(cv_image), threshU1=8,threshU2=20, threshV2=70)
        display_obstacles = cv2.cvtColor(self.vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in self.vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)

        try:
            self.obs_disp = np.copy(display_obstacles)
            self.image_pub.publish(self.bridge.cv2_to_imgmsg(display_obstacles, "bgr8"))
        except CvBridgeError as e:
            print(e)
            
    def start(self):
        while not rospy.is_shutdown():
            self.r.sleep()

In [ ]:
io = image_converter()

In [ ]:
tmp = cv2.convertScaleAbs(io.vboat.img, None,0,0.001)
plt.figure(2)
plt.imshow(io.vboat.umap_raw)
plt.show()

In [ ]:
disp = np.zeros_like(io.vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)

umap = io.vboat.umap_processed
umap.astype
xs = io.vboat.xBounds
ds = io.vboat.dbounds
obs = io.vboat.obstacles

# masks = []
# for win in io.vboat.obstacles:
#     mask = np.copy(disp)
#     cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
#     mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
#     masks.append(mask)
    

# # tmp = cv2.bitwise_and(io.vboat.umap_raw,io.vboat.umap_raw, mask=masks[0])
# tmp = cv2.bitwise_and(io.vboat.img,io.vboat.img, mask=masks[0])


# tmpMean = np.mean(tmp)
# # tmpMean = cv2.meanStdDev(io.vboat.umap_raw,mask=masks[0])
# print(tmpMean)

plt.figure(1)
plt.imshow(io.obs_disp)
plt.show()



plt.figure(3)
plt.imshow(io.vboat.umap_raw)
plt.show()

print(ds)

fx = 462.13797
b = 55
d = np.mean(ds[0])
# d = tmpMean
dist = ((fx*b)/d)*0.001

x0 = 1.177564+0.26
y0 = -0.067304

x1 = 4.000000
y1 = -1.000000

dx = x1 - x0
dy = y1 - y0

dd = np.sqrt(dx*dx + dy*dy)
dd, dist,d, obs[1]

In [ ]:
i = 1
dss = ds[i]
xs = [obs[i][0][0], obs[i][1][0]]
ys = [obs[i][0][1], obs[i][1][1]]

io.vboat.calculate_distance(umap,xs,dss, ys)

In [ ]:
bot = [0.444806, -0.559817]
obs = [2.000000, -2]
dx = bot[0] - obs[0]
dy = bot[1] - obs[1]
dist = np.sqrt(dx*dx + dy*dy)
print(dist)

In [ ]:
win = io.vboat.obstacles_umap[0]
print(win)
crop_img = io.vboat.umap_raw[win[0][1]:win[1][1], win[0][0]:win[1][0]]

tmpMean = np.mean(crop_img)
print(tmpMean)

plt.figure(3)
plt.imshow(crop_img)
plt.show()